In [ ]:
import pickle
from datetime import datetime

import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# target = "target_close"
# target = "close_percent_change_1"
target = "up_down_close_1"

In [ ]:
df = pd.read_parquet("../data/dataset.parquet")

In [ ]:
def normalize_column(data: pd.DataFrame, column: str | list) -> pd.DataFrame:
    if isinstance(column, str):
        data[column] = (data[column] - data[column].min()) / (data[column].max() - data[column].min())
    elif isinstance(column, list):
        for col in column:
            data[col] = (data[col] - data[col].min()) / (data[col].max() - data[col].min())
    return data

In [ ]:
def normalize_series(data: pd.Series) -> pd.Series:
    return (data - data.min()) / (data.max() - data.min())

In [ ]:
df.filter(like="splits")

In [ ]:
# columns to drop "dividends"
df = df.drop(columns=["dividends_event_name", "splits_event_name"])


In [ ]:
# # Matplotlib Graphs
# plt.figure(figsize=(24, 18))

# # Correlation Matrix
# corrilations = df.corr()
# sns.heatmap(corrilations, annot=False, cmap="coolwarm")
# plt.title("Correlation Matrix")
# plt.show()


In [ ]:
list(df.columns)

In [ ]:
df.filter(like="close_fib")

In [ ]:
df = normalize_column(df, ["rsi_30"])

In [ ]:
df["line_0.7"] = 0.7
df["line_0.5"] = 0.5
df["line_0.3"] = 0.3

In [ ]:
# only the last 365 days
# df = df.iloc[-720:]

In [ ]:
# More Graphs
plt.figure(figsize=(20, 8))
plt.plot(df["close"])
# plt.plot(df["close_bollinger_upper_7"])
# plt.plot(df["close_bollinger_upper_14"])
plt.plot(df["close_bollinger_upper_30"])
# plt.plot(df["close_bollinger_lower_7"])
# plt.plot(df["close_bollinger_lower_14"])


plt.plot(df["rsi_30"])
plt.plot(df["close_bollinger_lower_30"])
plt.plot(df["close_rolling_mean_30"])

# plt.plot(df["close_fib_0.382"])
plt.plot(df["close_fib_0.618"])


plt.title("Close Price")
plt.legend(["Close", "Upper 7", "Upper 14", "Upper 30", "Lower 7", "Lower 14", "Lower 30"])
plt.show()


In [ ]:
fig = plt.figure(figsize=(24, 10))
gs = fig.add_gridspec(2, hspace=0)
axs = gs.subplots(sharex=True, sharey=False)
fig.suptitle('Sharing both axes')

# df = df.iloc[-60:]


axs[0].plot(df["close"])
axs[0].plot(df["close_bollinger_upper_30"])
axs[0].plot(df["close_rolling_mean_30"])
axs[0].plot(df["close_bollinger_lower_30"])

axs[1].plot(df["rsi_30"])
axs[1].plot(df["line_0.7"])
axs[1].plot(df["line_0.5"])
axs[1].plot(df["line_0.3"])

In [ ]:
# df.set_index("date", inplace=True)

In [ ]:
df.filter(like="_fib_")

In [ ]:
import yfinance as yf
import pandas as pd


import plotly.graph_objects as go

# axs[0].plot(df["close"])
# axs[0].plot(df["close_bollinger_upper_30"])
# axs[0].plot(df["close_rolling_mean_30"])
# axs[0].plot(df["close_bollinger_lower_30"])

# only take the last 365 days
df = df.iloc[-365:]

fig = go.Figure(
    data=[
        go.Candlestick(
            x=df['date'],
            open=df['open'],
            high=df['high'],
            low=df['low'],
            close=df['close'],
            name='close candlestick'
        ),
        go.Scatter(
            x=df['date'],
            y=df['close'],
            mode='lines',
            line=go.scatter.Line(color='red'),
            name='close'
        ),
        go.Scatter(
            x=df['date'],
            y=df['close_rolling_mean_30'],
            mode='lines',
            line=go.scatter.Line(color='green'),
            name='MA 30'
        ),
        # go.Scatter(
        #     x=df['date'],
        #     y=df['close_bollinger_upper_30'],
        #     mode='lines',
        #     line=go.scatter.Line(color='blue'),
        #     name='Upper 30'
        # ),
        # go.Scatter(
        #     x=df['date'],
        #     y=df['close_bollinger_lower_30'],
        #     mode='lines',
        #     line=go.scatter.Line(color='blue'),
        #     name='Lower 30'
        # ),


        go.Scatter(
            x=df['date'],
            y=df['macd_12_26'],
            mode='lines',
            line=go.scatter.Line(color='yellow'),
            name='macd_12_26'
        ),
        go.Scatter(
            x=df['date'],
            y=df['macd_hist_12_26_9'],
            mode='lines',
            line=go.scatter.Line(color='purple'),
            name='macd_hist_12_26_9'
        ),
        # go.Scatter(
        #     x=df['date'],
        #     y=df['lead_1_30'],
        #     mode='lines',
        #     line=go.scatter.Line(color='orange'),
        #     name='lead_1_30'
        # ),
        # go.Scatter(
        #     x=df['date'],
        #     y=df['lead_2_30'],
        #     mode='lines',
        #     line=go.scatter.Line(color='orange'),
        #     name='lead_2_30'
        # ),

    # data[f"short_ema_{short_window}"] = data[column].ewm(span=short_window, adjust=False).mean()
    # data[f"long_ema_{long_window}"] = data[column].ewm(span=long_window, adjust=False).mean()
    # data[f"macd_{short_window}_{long_window}"] = data[f"short_ema_{short_window}"] - data[f"long_ema_{long_window}"]
    # data[f"signal_{signal_window}"] = data[f"macd_{short_window}_{
    #     long_window}"].ewm(span=signal_window, adjust=False).mean()
    # data[f"macd_hist_{short_window}_{long_window}_{signal_window}"] = data[f"macd_{
    #     short_window}_{long_window}"] - data[f"signal_{signal_window}"]


    ]
)

fig.update_layout(
    xaxis_rangeslider_visible=True,
    autosize=False,
    margin=dict(l=20, r=20, t=20, b=20),
    width=1600,
    height=800,
    paper_bgcolor="LightSteelBlue",
)

fig.show()

In [ ]:
import yfinance as yf
import pandas as pd


import plotly.graph_objects as go
from plotly.subplots import make_subplots


# only take the last 365 days
df = df.iloc[-365:]

fig = make_subplots(rows=1, cols=1, shared_xaxes=True, vertical_spacing=0.02)


fig.append_trace(
    go.Candlestick(
        x=df['date'],
        open=df['open'],
        high=df['high'],
        low=df['low'],
        close=df['close'],
        name='close candlestick'
    ),
    row=1, col=1
)

fig.append_trace(
    go.Scatter(
        x=df['date'],
        y=df['close'],
        mode='lines',
        line=go.scatter.Line(color='red'),
        name='close'
    ),

    row=1, col=1
)
fig.append_trace(
    go.Scatter(
        x=df['date'],
        y=df['close_rolling_mean_30'],
        mode='lines',
        line=go.scatter.Line(color='green'),
        name='MA 30'
    ),
    row=1, col=1
)

fig.append_trace(
    go.Scatter(
        x=df['date'],
        y=df['signal_9'],
        mode='lines',
        line=go.scatter.Line(color='green'),
        name='MA Signal'
    ),
    row=1, col=1
)

# close_bollinger_upper_30
fig.append_trace(
    go.Scatter(
        x=df['date'],
        y=df['close_bollinger_upper_30'],
        mode='lines',
        line=go.scatter.Line(color='blue'),
        name='Upper 30'
    ),
    row=1, col=1
)
fig.append_trace(
    go.Scatter(
        x=df['date'],
        y=df['close_bollinger_lower_30'],
        mode='lines',
        line=go.scatter.Line(color='blue'),
        name='Lower 30'
    ),
    row=1, col=1
)




# fig.append_trace(
#     go.Scatter(
#         x=df['date'],
#         y=df['rsi_30'],
#         mode='lines',
#         line=go.scatter.Line(color='green'),
#         name='rsi_30'
#     ),
#     row=2, col=1,
# )



# fig.append_trace(
#     go.Scatter(
#         x=df['date'],
#         y=df['short_ema_50'],
#         mode='lines',
#         line=go.scatter.Line(color='red'),
#         name='short_ema_50'
#     ),
#     row=3, col=1,
# )
# fig.append_trace(
#     go.Scatter(
#         x=df['date'],
#         y=df['long_ema_200'],
#         mode='lines',
#         line=go.scatter.Line(color='black'),
#         name='long_ema_200'
#     ),
#     row=3, col=1,
# )

# fig.append_trace(
#     go.Scatter(
#         x=df['date'],
#         y=df['macd_hist_12_26_9'],
#         mode='lines',
#         line=go.scatter.Line(color='purple'),
#         name='macd_hist_12_26_9'
#     ),
#     row=3, col=1,
# )

# close_fib_0.236	close_fib_0.382	close_fib_0.618	close_fib_1.0

fig.update_layout(
    xaxis_rangeslider_visible=True,
    autosize=True,
    margin=dict(l=20, r=20, t=20, b=20),
    width=1600,
    height=800,
    paper_bgcolor="LightSteelBlue",
)

fig.show()